<a href="https://colab.research.google.com/github/SiddhiMahajan594/Electron-Cipher/blob/main/Electron_Cipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install pycryptodome ipywidgets

In [70]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA256
import base64
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
import numpy as np
from random import randint

Orbital Config

In [71]:
orbital_order = [('s', 2), ('p', 6), ('d', 10), ('f', 14)]

Key Derivation

In [72]:
def derive_key_from_password(password, salt=None):
    if salt is None:
        salt = get_random_bytes(16)
    password_bytes = password.encode() if isinstance(password, str) else password
    key = PBKDF2(password_bytes, salt, dkLen=32, count=100000)
    return key, salt


Generate Dynamic S-boxes

In [73]:
import numpy as np
import random
import hashlib

# -- Fitness placeholder --
def calculate_non_linearity(sbox):
    return np.random.randint(100, 300)

def calculate_differential_uniformity(sbox):
    return np.random.randint(2, 16)

def fitness(sbox):
    nl = calculate_non_linearity(sbox)
    du = calculate_differential_uniformity(sbox)
    return nl - 5 * du

# -- Deterministic GA S-box generator --
def generate_dynamic_sboxes_using_ga(password):
    # Hash password -> int seed
    seed = int(hashlib.sha256(password.encode()).hexdigest(), 16) % (2**32)
    random.seed(seed)
    np.random.seed(seed)

    def generate_sbox_ga(length, generations=50, population_size=30):
        population = [list(np.random.permutation(length)) for _ in range(population_size)]
        for generation in range(generations):
            population.sort(key=fitness, reverse=True)
            new_population = population[:5]
            while len(new_population) < population_size:
                p1, p2 = random.sample(population[:15], 2)
                if length > 2:
                    cut = random.randint(1, length - 2)
                else:
                    cut = 1
                child = p1[:cut] + [x for x in p2 if x not in p1[:cut]]
                if random.randint(0, 10) < 3:
                    i, j = random.sample(range(length), 2)
                    child[i], child[j] = child[j], child[i]
                new_population.append(child)
            population = new_population
        best_sbox = population[0]
        return best_sbox

    dynamic_sboxes = {}
    orbital_lengths = {'s': 2, 'p': 6, 'd': 10, 'f': 14}
    for orbital, length in orbital_lengths.items():
        sbox = generate_sbox_ga(length)
        dynamic_sboxes[orbital] = sbox
    return dynamic_sboxes


def generate_inverse_sboxes(sboxes):
    inv_sboxes = {}
    for orbital, sbox in sboxes.items():
        inv = [0] * len(sbox)
        for i, val in enumerate(sbox):
            inv[val] = i
        inv_sboxes[orbital] = inv
    return inv_sboxes

Orbital Permutation

In [74]:
def orbital_permute(binary, sboxes):
    result = ''
    idx = 0
    order_idx = 0
    mappings = []

    while idx < len(binary):
        orbital, bits = orbital_order[order_idx % len(orbital_order)]
        sbox = sboxes[orbital]
        segment = binary[idx:idx + bits]
        if len(segment) < bits:
            segment = segment.ljust(bits, '0')
        permuted = ''.join(segment[sbox[i]] if i < len(segment) else '0' for i in range(bits))
        result += permuted
        mappings.append((orbital, segment, permuted))
        idx += bits
        order_idx += 1

    return result, mappings


Orbital Unpermutation

In [75]:
def orbital_unpermute(binary, inv_sboxes):
    result = ''
    idx = 0
    order_idx = 0

    while idx < len(binary):
        orbital, bits = orbital_order[order_idx % len(orbital_order)]
        inv_sbox = inv_sboxes[orbital]
        segment = binary[idx:idx + bits]
        if len(segment) < bits:
            segment = segment.ljust(bits, '0')
        unpermuted = ['0'] * bits
        for i in range(bits):
            src_idx = inv_sbox[i]
            if src_idx < len(segment):
                unpermuted[i] = segment[src_idx]
        result += ''.join(unpermuted)
        idx += bits
        order_idx += 1

    return result


Binary Conversion Helpers

In [76]:
def binary_to_bytes(binary):
    padding = (8 - len(binary) % 8) % 8
    padded = binary + '0' * padding
    byte_array = bytearray()
    for i in range(0, len(padded), 8):
        byte = padded[i:i+8]
        byte_array.append(int(byte, 2))
    return bytes(byte_array)

def bytes_to_binary(byte_data, length=None):
    binary = ''.join(format(b, '08b') for b in byte_data)
    if length is not None:
        binary = binary[:length]
    return binary


Feistel Function

In [77]:
import random

def feistel_round(left, right, subkey, orbital):
    # Generate a non-linear function based on orbital
    def f_function(r, k):
        r_int = int(r, 2)
        k_int = int(k, 2)
        xor_result = r_int ^ k_int
        if orbital == 's':
            return format((xor_result ^ (xor_result << 1)) & 0xFF, '08b')
        elif orbital == 'p':
            return format((xor_result ^ (xor_result >> 1)) & 0xFF, '08b')
        elif orbital == 'd':
            return format(((xor_result << 2) ^ (xor_result >> 2)) & 0xFF, '08b')
        elif orbital == 'f':
            return format(((xor_result & 0xAA) | (~xor_result & 0x55)) & 0xFF, '08b')
        return format(xor_result, '08b')

    f_out = f_function(right, subkey)
    new_right = format(int(left, 2) ^ int(f_out, 2), '08b')
    return right, new_right

def feistel_encrypt_block(block, rounds=4, key="10101010", orbital='s'):
    half = len(block) // 2
    left, right = block[:half], block[half:]
    for i in range(rounds):
        subkey = format((int(key, 2) + i) % 256, '08b')
        left, right = feistel_round(left, right, subkey, orbital)
    return left + right

def feistel_decrypt_block(block, rounds=4, key="10101010", orbital='s'):
    half = len(block) // 2
    left, right = block[:half], block[half:]
    round_keys = [format((int(key, 2) + i) % 256, '08b') for i in range(rounds)][::-1]
    for subkey in round_keys:
        right, left = feistel_round(right, left, subkey, orbital)
    return left + right

def orbital_feistel_mix(binary, orbital='s', rounds=4):
    padded_binary = binary
    if len(binary) % 16 != 0:
        padded_binary = binary + '0' * (16 - len(binary) % 16)

    mixed = ''
    for i in range(0, len(padded_binary), 16):
        block = padded_binary[i:i+16]
        mixed += feistel_encrypt_block(block, rounds=rounds, orbital=orbital)

    return mixed

def orbital_feistel_unmix(binary, orbital='s', rounds=4):
    unmixed = ''
    for i in range(0, len(binary), 16):
        block = binary[i:i+16]
        unmixed += feistel_decrypt_block(block, rounds=rounds, orbital=orbital)

    return unmixed


Encryption & Decryption

In [78]:
def encrypt_with_orbitals(plaintext, password):
    key, salt = derive_key_from_password(password)
    iv = get_random_bytes(16)
    plaintext_bytes = plaintext.encode()
    binary = ''.join(format(byte, '08b') for byte in plaintext_bytes)
    print(f"Plaintext bits:  {binary}")

    # Apply orbital Feistel mixing
    feistel_mixed_binary = orbital_feistel_mix(binary, orbital='p', rounds=4)
    print(f"Feistel Mixed:   {feistel_mixed_binary}")

    # Permute using orbital S-boxes
    sboxes = generate_dynamic_sboxes_using_ga(password)
    permuted_binary, mappings = orbital_permute(feistel_mixed_binary, sboxes)
    print(f"Permuted bits:   {permuted_binary}")

    byte_data = binary_to_bytes(permuted_binary)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    encrypted = cipher.encrypt(pad(byte_data, AES.block_size))
    length_bytes = len(binary).to_bytes(4, byteorder='big')
    result = salt + iv + length_bytes + encrypted
    return base64.b64encode(result).decode(), mappings, binary, permuted_binary

def decrypt_with_orbitals(ciphertext, password):
    try:
        raw = base64.b64decode(ciphertext)
        salt = raw[:16]
        iv = raw[16:32]
        original_length = int.from_bytes(raw[32:36], byteorder='big')
        encrypted = raw[36:]
    except Exception as e:
        print(f"Error parsing ciphertext: {e}")
        return ""
    key, _ = derive_key_from_password(password, salt)
    try:
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted_padded = cipher.decrypt(encrypted)
        decrypted = unpad(decrypted_padded, AES.block_size)
    except Exception as e:
        print(f"Decryption error: {e}")
        return ""
    decrypted_binary = bytes_to_binary(decrypted)
    print(f"Decrypted permuted bits: {decrypted_binary}")

    sboxes = generate_dynamic_sboxes_using_ga(password)
    inv_sboxes = generate_inverse_sboxes(sboxes)
    original_permuted_binary = orbital_unpermute(decrypted_binary, inv_sboxes)

    # Reverse Feistel mixing
    original_binary = orbital_feistel_unmix(original_permuted_binary, orbital='p', rounds=4)
    original_binary = original_binary[:original_length]
    print(f"Recovered bits:          {original_binary}")

    try:
        padded_binary = original_binary
        if len(padded_binary) % 8 != 0:
            padded_binary = padded_binary + '0' * (8 - len(padded_binary) % 8)
        bytes_array = bytearray()
        for i in range(0, len(padded_binary), 8):
            if i + 8 <= len(padded_binary):
                byte_val = int(padded_binary[i:i+8], 2)
                bytes_array.append(byte_val)
        return bytes(bytes_array).decode('utf-8', errors='replace')
    except Exception as e:
        print(f"Error decoding plaintext: {e}")
        return ""


Visualization Functions

In [79]:
def visualize_orbital_permutation(mappings):
    fig, ax = plt.subplots(figsize=(14, len(mappings)))
    ax.set_xlim(0, max(len(before) for _, before, _ in mappings) + 1)
    ax.set_ylim(-1, len(mappings) * 2.5)
    ax.axis('off')
    vertical_gap = 1.4
    after_offset = 0.4
    for i, (orbital, before, after) in enumerate(mappings):
        y_base = len(mappings) * 2.5 - i * 2.5 - 1
        ax.text(-0.5, y_base + vertical_gap / 2, f"{orbital}", fontsize=10, weight='bold', va='center')
        for j in range(len(before)):
            bx = j
            before_y = y_base + vertical_gap / 2
            arrow_start_y = before_y - 0.1
            arrow_length = -vertical_gap + 0.2
            after_y = arrow_start_y + arrow_length - after_offset
            ax.text(bx, before_y, before[j], ha='center', fontsize=9)
            ax.arrow(bx, arrow_start_y, 0, arrow_length, head_width=0.1, head_length=0.1, fc='gray', ec='gray')
            ax.text(bx, after_y, after[j], ha='center', fontsize=9, color='blue')
    plt.title("Orbital-Based Bit Permutation (Top: Before, Bottom: After)", fontsize=14)
    plt.tight_layout()
    plt.show()

def visualize_binary(binary, title='Bit Structure'):
    fig, ax = plt.subplots(figsize=(min(0.5 * len(binary), 24), 2))
    ax.set_xlim(0, len(binary))
    ax.set_ylim(0, 1)
    ax.axis('off')
    for i, bit in enumerate(binary):
        color = 'blue' if bit == '1' else 'lightgray'
        ax.scatter(i, 0.5, c=color, s=300, edgecolors='black', zorder=2)
        ax.text(i, 0.5, bit, ha='center', va='center', fontsize=8, zorder=3)
    plt.title(title, fontsize=16)
    plt.tight_layout()
    plt.show()


UI + Logic (Widgets)

In [80]:
text_input = widgets.Text(
    value='Cyber Security',
    placeholder='Enter text to encrypt',
    description='Plaintext:',
    layout=widgets.Layout(width='400px')
)

password_input = widgets.Password(
    value='hello',
    description='Password:',
    layout=widgets.Layout(width='400px')
)

encrypt_button = widgets.Button(description="Encrypt & Decrypt")
output = widgets.Output()

def run_encryption(b):
    output.clear_output()
    with output:
        plaintext = text_input.value
        password = password_input.value
        if not plaintext or not password:
            print("Please enter both plaintext and password.")
            return

        print("\n--- Encryption ---")
        ciphertext, mappings, orig_bits, perm_bits = encrypt_with_orbitals(plaintext, password)

        # 🔍 Show dynamic S-box and inverse S-box table
        key, _ = derive_key_from_password(password)
        sboxes = generate_dynamic_sboxes_using_ga(password)
        inv_sboxes = generate_inverse_sboxes(sboxes)

        print("\n=== Dynamic S-Box Table ===")
        for orbital in ['s', 'p', 'd', 'f']:
            print(f"{orbital.upper()} S-box:     {sboxes[orbital]}")
            print(f"{orbital.upper()} Inv S-box: {inv_sboxes[orbital]}")

        print("\nCiphertext (Base64):")
        print(ciphertext)

        print("\n--- Decryption ---")
        recovered = decrypt_with_orbitals(ciphertext, password)
        print("\nRecovered Plaintext:")
        print(recovered)

        print("\n--- Bit Visualizations ---")
        visualize_binary(orig_bits, 'Original Binary Bits')
        visualize_binary(perm_bits, 'Permuted Bits (After Orbital S-box)')
        visualize_orbital_permutation(mappings)

encrypt_button.on_click(run_encryption)
display(text_input, password_input, encrypt_button, output)


Text(value='Cyber Security', description='Plaintext:', layout=Layout(width='400px'), placeholder='Enter text t…

Password(description='Password:', layout=Layout(width='400px'))

Button(description='Encrypt & Decrypt', style=ButtonStyle())

Output()